In [1]:
!pip install azure-storage-file-datalake azure-identity pyodbc

DEPRECATION: Loading egg at c:\users\usuario\appdata\local\programs\python\python311\lib\site-packages\nester-1.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


HAY QUE TENER INSTALADO EN WINDOWS EL DRIVER ODBC PARA BASES DE DATOS SQL QUE SE PUEDE DESCARGAR DE ESTE LINK
https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15#download-for-windows
Igual lo subi al repo de GitHub es el archivo msodbcsql.msi en la carpeta Utils

In [2]:
from azure.identity import InteractiveBrowserCredential

credential = InteractiveBrowserCredential()

In [3]:
import os

In [6]:
from azure.storage.filedatalake import (
    DataLakeServiceClient,
    DataLakeDirectoryClient,
    FileSystemClient
)

In [29]:
#cadena de conexion al punto de datos SQL del datalake de fabric MLModel
sql_endpoint = "nwn7f7ze6vtuxen5age454nhca-p44tzcajmseujpwnyirs57khce.datawarehouse.fabric.microsoft.com" # copy and paste the SQL endpoint from any of the Lakehouses or Warehouses in your Fabric Workspace
database = "MLModel" # copy and paste the name of the Lakehouse or Warehouse you want to connect to
#"https://onelake.dfs.fabric.microsoft.com/883c397f-6409-4489-becd-c2232efd4711/05bfc39e-219f-42de-9013-1ee05672af29/Tables"
connection_string = f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_endpoint},1433;Database=f{database};Encrypt=Yes;TrustServerCertificate=No"

In [8]:
import struct
from itertools import chain, repeat
import pyodbc

In [9]:
# prepare the access token - Aquí se abre una ventana del browser y pide ingresar a nuestra cuenta de fabric para autenticar

token_object = credential.get_token("https://database.windows.net//.default") # Retrieve an access token valid to connect to SQL databases
token_as_bytes = bytes(token_object.token, "UTF-8") # Convert the token to a UTF-8 byte string


In [30]:
encoded_bytes = bytes(chain.from_iterable(zip(token_as_bytes, repeat(0)))) # Encode the bytes to a Windows byte string
token_bytes = struct.pack("<i", len(encoded_bytes)) + encoded_bytes # Package the token into a bytes object
attrs_before = {1256: token_bytes}  # Attribute pointing to SQL_COPT_SS_ACCESS_TOKEN to pass access token to the driver


# build the connection

connection = pyodbc.connect(connection_string, attrs_before=attrs_before)


In [31]:
cursor = connection.cursor()
cursor.execute("SELECT * FROM sys.tables")
rows = cursor.fetchall()
print(rows) # this will print all the tables available in the lakehouse or warehouse

[('circuitswithimages', 359008360, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2024, 9, 3, 12, 5, 53, 317000), datetime.datetime(2024, 9, 3, 12, 5, 53, 317000), False, False, False, 0, None, 10, False, True, False, False, False, False, False, 0, False, False, 0, 'TABLE', False, False, 0, 'SCHEMA_AND_DATA', 0, 'NON_TEMPORAL_TABLE', None, False, False, None, None, None, False, False, -1, -1, 'INFINITE', 0, 'NON_LEDGER_TABLE', None, False), ('driverswithimages', 375008417, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2024, 9, 3, 12, 5, 55, 110000), datetime.datetime(2024, 9, 3, 12, 5, 55, 110000), False, False, False, 0, None, 10, False, True, False, False, False, False, False, 0, False, False, 0, 'TABLE', False, False, 0, 'SCHEMA_AND_DATA', 0, 'NON_TEMPORAL_TABLE', None, False, False, None, None, None, False, False, -1, -1, 'INFINITE', 0, 'NON_LEDGER_TABLE', None, False), ('constructorswithimages', 391008474, None, 1, 0, 'U ', 'USER_TABLE', datetime.datetime(2024, 9, 3, 12, 5,

In [24]:
def generate_dataset(yearfrom,yearuntil,positionfrom,positionuntil):
    cursor.execute( f"""
    SELECT ra.raceId, ra.year,ra.round,ra.circuitId,re.driverId,re.constructorId,
    re.grid,TRY_CAST(re.position as int) as position,re.points,re.statusId,st.status,
    re.laps,re.fastestLap,re.fastestLapTime
    FROM races ra 
    JOIN results re ON ra.raceId = re.raceId
    JOIN status st ON st.statusId= re.statusId
    WHERE position >= {positionfrom} AND position <= {positionuntil} AND ra.year >= {yearfrom} AND ra.year <= {yearuntil} 
    ORDER BY ra.year desc,ra.round, position""")
    df= cursor.fetchall()
    return df

TUTORIAL DE CONEXION DESDE VSCODE AL DATALAKE DE FABRIC A TRAVES DE ODBC:
https://debruyn.dev/2023/connect-to-fabric-lakehouses-warehouses-from-python-code/